# DQN for text plagiarism detection

In [44]:
import json
import math
import os
from functools import partial
from typing import Literal, Protocol

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from cycler import cycler
from torch.utils.data import Dataset
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
from tqdm import tqdm

In [45]:
EpsilonStrategy = Literal["cos"] | Literal["line"] | Literal["exp"]


class EpsilonCallable(Protocol):
    def __call__(
        self,
        epoch: int,
        epsilon_start: float,
        epsilon_end: float,
        last_epoch: int,
        epsilon_const: float,
    ) -> float:
        ...

In [46]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# DEVICE = torch.device("cpu")
DEVICE

device(type='cuda')

In [47]:
LOG_PATH_PREFIX = "../logs/dqn/"

## Hyper parameters

In [48]:
RL_GAMMA = 1.0

GLOVE_DIM = 300
TRAIN_SIZE = "md"
TEST_SIZE = "sm"

EMBED_DIM = GLOVE_DIM
LSTM_LAYERS = 1
LSTM_H_DIM = EMBED_DIM
RNET_DROPOUT = 0.5

ENV_GAMMA = 0.1

DQN_LR = 0.005
RNET_LR = 0.005
SRM_LR = 0.005

DQN_SYNC_PERIOD = 2
DQN_CLIP_GRAD = 0.0

PRETRAIN_SRM_RNET_EPOCHS = 200
PRETRAIN_DQN_EPOCHS = 100

EPISODES_BATCH = 10
PRETRAIN_SRM_RNET_BATCH = EPISODES_BATCH

EPSILON_START = 1.0
EPSILON_END = 0.01
EPSILON_STRATEGY: EpsilonStrategy = "cos"
EPSILON_CONSTANT = 50  # 1.4

FEATURES = ""
COMMENTS = ""

EPOCHS = 1500

In [49]:
EXPERIMENT_NAME = "23_04_2"
EPOCH = 1140

In [50]:
def load_from_experiments(experiment: str) -> None:
    global \
        RL_GAMMA, \
        GLOVE_DIM, \
        TRAIN_SIZE, \
        TEST_SIZE, \
        EMBED_DIM, \
        LSTM_LAYERS, \
        LSTM_H_DIM, \
        RNET_DROPOUT, \
        ENV_GAMMA, \
        DQN_LR, \
        RNET_LR, \
        SRM_LR, \
        DQN_SYNC_PERIOD, \
        DQN_CLIP_GRAD, \
        PRETRAIN_SRM_RNET_EPOCHS, \
        PRETRAIN_DQN_EPOCHS, \
        EPISODES_BATCH, \
        PRETRAIN_SRM_RNET_BATCH, \
        EPSILON_START, \
        EPSILON_END, \
        EPSILON_STRATEGY, \
        EPSILON_CONSTANT, \
        FEATURES, \
        COMMENTS
    with open(os.path.join(".", LOG_PATH_PREFIX, experiment, "configs.json"), "r") as f:
        hyper_dict = json.load(f)
    RL_GAMMA = hyper_dict["RL_GAMMA"]

    GLOVE_DIM = hyper_dict["GLOVE_DIM"]
    TRAIN_SIZE = hyper_dict["TRAIN_SIZE"]
    TEST_SIZE = hyper_dict["TEST_SIZE"]
    EMBED_DIM = hyper_dict["EMBED_DIM"]
    LSTM_LAYERS = hyper_dict["LSTM_LAYERS"]
    LSTM_H_DIM = hyper_dict["LSTM_H_DIM"]
    RNET_DROPOUT = hyper_dict["RNET_DROPOUT"]
    ENV_GAMMA = hyper_dict["ENV_GAMMA"]
    DQN_LR = hyper_dict["DQN_LR"]
    RNET_LR = hyper_dict["RNET_LR"]
    SRM_LR = hyper_dict["SRM_LR"]
    DQN_SYNC_PERIOD = hyper_dict["DQN_SYNC_PERIOD"]
    DQN_CLIP_GRAD = hyper_dict["DQN_CLIP_GRAD"]
    PRETRAIN_SRM_RNET_EPOCHS = hyper_dict["PRETRAIN_SRM_RNET_EPOCHS"]
    PRETRAIN_DQN_EPOCHS = hyper_dict["PRETRAIN_DQN_EPOCHS"]
    EPISODES_BATCH = hyper_dict["EPISODES_BATCH"]
    PRETRAIN_SRM_RNET_BATCH = hyper_dict["PRETRAIN_SRM_RNET_BATCH"]
    EPSILON_START = hyper_dict["EPSILON_START"]
    EPSILON_END = hyper_dict["EPSILON_END"]
    EPSILON_STRATEGY = hyper_dict["EPSILON_STRATEGY"]
    EPSILON_CONSTANT = hyper_dict["EPSILON_CONSTANT"]
    FEATURES = hyper_dict["FEATURES"]
    COMMENTS = hyper_dict["COMMENTS"]
    print(hyper_dict)

## Epsilon Strategies

In [51]:
def epsilon_cos(
    epoch: int,
    epsilon_start: float,
    epsilon_end: float,
    last_epoch: int,
    epsilon_const=50.0,
) -> float:
    return (
        epsilon_start
        * (np.cos(epsilon_const * epoch - epsilon_const) + 1)
        * np.exp(-4 / last_epoch * (epoch - 1))
        / 2
    )


def epsilon_line(
    epoch: int,
    epsilon_start: float,
    epsilon_end: float,
    last_epoch: int,
    epsilon_const=20.0,
) -> float:
    return -np.emath.logn(
        epsilon_const,
        (
            np.float_power(epsilon_const, -epsilon_start) * (epoch - last_epoch)
            + np.float_power(epsilon_const, -epsilon_end) * (1 - epoch)
        )
        / (1 - last_epoch),
    )


def epsilon_exp(
    epoch: int,
    epsilon_start: float,
    epsilon_end: float,
    last_epoch: int,
    epsilon_const: float = 1.4,
) -> float:
    return epsilon_start * np.exp(
        np.log(epsilon_end / epsilon_start)
        * epsilon_const
        * (epoch - 1)
        / (last_epoch - 1)
    )


EPSILON_STRATEGY_DICT: dict[EpsilonStrategy, EpsilonCallable] = {
    "cos": epsilon_cos,
    "line": epsilon_line,
    "exp": epsilon_exp,
}

## Utilities

In [52]:
PREDEFINED_COLORS = [
    "#ffa500",
    "#c83cbc",
    "#1c1c84",
    "#ff0000",
    "#08a4a7",
    "#008000",
]


def get_plots(
    data_dict: dict,
    plots: list[tuple[dict, dict]],
    title: str = "",
    ylim=None,
    row_plots: int = 1,
    plot_width: float = 8,
    plot_height: float = 4,
    use_rainbow: bool = False,
    use_common_legend: bool = False,
    adjust: bool = False,
):
    num_plots = len(plots)
    num_entities = max([len(x[1]) for x in plots]) + 1
    if use_rainbow:
        num_colors = num_entities
        cm = plt.get_cmap("gist_rainbow")
        colors = [cm(1.0 * i / num_colors) for i in range(num_colors)]
    else:
        colors = PREDEFINED_COLORS

    style_cycler = cycler(linestyle=["-", "--", ":", "-."]) * cycler(color=colors)
    column_plots = math.ceil(num_plots / row_plots)

    fig, axs = plt.subplots(
        column_plots,
        row_plots,
        figsize=(plot_width * row_plots, plot_height * column_plots),
    )

    if len(title) > 0:
        fig.suptitle(title, fontsize=14, y=1)
    axs_list = [axs] if column_plots * row_plots == 1 else list(axs.flat)

    for ax in axs_list:
        ax.grid()
        ax.set_prop_cycle(style_cycler)
        if ylim is not None:
            ax.set_ylim(top=ylim)
        ax.set_visible(False)

    for ax, (p1, p2) in zip(axs_list, plots):
        ax.set_visible(True)

        ax.set_title(f"{p2['axis_name']} over {p1['axis_name']}")
        ax.set(xlabel=p1["axis_label"], ylabel=p2["axis_label"])

        if p1.get("log", False):
            ax.set_xscale("log")
        if p2.get("log", False):
            ax.set_yscale("log")

        x_values = data_dict[p1.get("ref", None) or p1["axis_name"]]

        p2vs = p2.get("values", [])
        if len(p2vs) == 0:
            y_values = data_dict[p2.get("ref", None) or p2["axis_name"]]
            ax.plot(x_values, y_values, label=p2["axis_name"])
            ax.scatter(x_values[-1], y_values[-1], s=15)
            continue

        for p2v in p2vs:
            y_values = data_dict[p2v.get("ref", None) or p2v["name"]]

            try:
                iter(y_values)
                ax.plot(x_values, y_values, label=p2v["name"])
                ax.scatter(x_values[-1], y_values[-1], s=15)
            except TypeError:
                ax.plot(x_values, [y_values] * len(x_values), label=p2v["name"])

    if use_common_legend:
        lines_labels = [axs_list[0].get_legend_handles_labels()]
        lines, labels = [sum(x, []) for x in zip(*lines_labels)]
        fig.legend(
            lines,
            labels,
            scatterpoints=1,
            markerscale=3,
            loc="outside lower center",
            ncol=min(6, num_entities),
            bbox_to_anchor=(0.5, -0.05),
        )
    else:
        if num_entities > 1:
            for ax, _ in zip(axs_list, plots):
                ax.legend()

    plt.tight_layout()
    if adjust:
        plt.subplots_adjust(
            top=1 - 0.1 / (num_plots**0.5), bottom=0.12 / (num_plots**2), hspace=0.15
        )

    return fig


def draw_plots(
    data_dict: dict,
    plots: list[tuple[dict, dict]],
    title: str = "",
    ylim=None,
    row_plots: int = 1,
    plot_width: float = 8,
    plot_height: float = 4,
    use_rainbow: bool = False,
    use_common_legend: bool = False,
    adjust: bool = False,
):
    get_plots(
        data_dict,
        plots,
        title,
        ylim,
        row_plots,
        plot_width,
        plot_height,
        use_rainbow,
        use_common_legend,
        adjust,
    )
    plt.show()

## Loader

In [53]:
def load_model(name: str, model):
    path = os.path.join(
        ".", LOG_PATH_PREFIX, EXPERIMENT_NAME, "best", str(EPOCH), "models", name
    )
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model_state_dict"])

## Dataset

In [54]:
tokenizer = get_tokenizer("basic_english")
global_vectors = GloVe(dim=GLOVE_DIM, cache="../data")


def text_pipeline(x):
    return global_vectors.get_vecs_by_tokens(tokenizer(x), lower_case_backup=True)


def tokenized_pipeline(x):
    return global_vectors.get_vecs_by_tokens(x, lower_case_backup=True)

In [55]:
def read_from_disk(path: str) -> np.ndarray:
    return pd.read_csv(path).to_numpy()

In [56]:
class PlagiarismDataset(Dataset):
    def __init__(self, data: np.ndarray):
        targets, candidates, scores = [], [], []

        for target, candidate, score in data:
            targets.append(tokenizer(target))
            candidates.append(tokenizer(candidate))
            scores.append(score)

        self.targets = targets
        self.candidates = candidates
        self.scores = np.array(scores).astype(np.float16)

    def __len__(self):
        return len(self.scores)

    def __getitem__(
        self, idx
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, list, list]:
        return (
            tokenized_pipeline(self.targets[idx]).to(DEVICE),
            tokenized_pipeline(self.candidates[idx]).to(DEVICE),
            torch.tensor([self.scores[idx]]).float().to(DEVICE),
            self.targets[idx],
            self.candidates[idx],
        )

In [57]:
test_data = PlagiarismDataset(
    read_from_disk(f"../generated/datasets/test_{TEST_SIZE}.csv")
)

print(f"{len(test_data)=}")

len(test_data)=147


## RNet & SRModel

In [58]:
class RNetNN(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int = 128) -> None:
        super(RNetNN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(RNET_DROPOUT),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.net(x)


class RNet:
    def __init__(self, lr: float = 1e-2, device=DEVICE):
        self.net = RNetNN(2 * LSTM_LAYERS * LSTM_H_DIM, 1).to(device)
        self.loss_fn = F.mse_loss
        self.optimizer = optim.Adam(
            self.net.parameters(),
            lr=lr,
        )

    def __call__(self, data: torch.Tensor, grad: bool = True) -> torch.Tensor:
        if grad:
            return self.net(data)
        with torch.no_grad():
            return self.net(data)


class SRModelNN(nn.Module):
    def __init__(
        self, input_dim: int, hidden_size: int, num_layers: int = LSTM_LAYERS
    ) -> None:
        super(SRModelNN, self).__init__()

        self.net = nn.LSTM(
            input_dim,
            hidden_size,
            num_layers=num_layers,
            bidirectional=False,
        ).to(DEVICE)

    def forward(self, *x):
        return self.net(*x)


class SRModel:
    def __init__(self, lr: float = 1e-2, device=DEVICE):
        self.net = SRModelNN(EMBED_DIM, LSTM_H_DIM).to(device)
        self.optimizer = optim.Adam(
            self.net.parameters(),
            lr=lr,
        )

        self.device = device

    def __call__(self, *data, grad: bool = True) -> torch.Tensor:
        if grad:
            return self.net(*data)
        with torch.no_grad():
            return self.net(*data)

    def call_batch(self, data, grad: bool = True) -> torch.Tensor:
        cat_data = torch.cat(data) if type(data) == list else data
        cat_data = cat_data.view(len(data), 1, -1)
        h_c = (
            torch.zeros(LSTM_LAYERS, 1, LSTM_H_DIM).to(self.device),
            torch.zeros(LSTM_LAYERS, 1, LSTM_H_DIM).to(self.device),
        )
        out, _ = self.__call__(cat_data, h_c, grad=grad)
        return out[-1].flatten()

In [59]:
def update_sample(
    srm: SRModel,
    rnet: RNet,
    target: list[torch.Tensor],
    candidate: list[torch.Tensor],
    train_srm: bool = True,
    train_rnet: bool = True,
):
    srm_out_target = srm.call_batch(target, train_srm)
    srm_out_candidate = srm.call_batch(candidate, train_srm)

    rnet_out = rnet(
        torch.cat([srm_out_target, srm_out_candidate]).view(1, -1), grad=train_rnet
    )

    return rnet_out.squeeze(-1)

## Environment

In [60]:
class Env:
    def _get_state(self) -> torch.Tensor:
        token = self.data[self.sentence_idx][self.token_idx]
        return torch.cat(
            [
                self.hs[self.sentence_idx].flatten(),
                self.cs[self.sentence_idx].flatten(),
                token,
            ]
        ).to(DEVICE)

    def _get_reward(self) -> float:
        if not self.is_terminal():
            return 0.0

        self.statistics_dict["Deletions ratio"] = (
            self.statistics_dict["Deletions"] / self.total_words
        )

        # Case when agent removes the entire sequence
        if self.statistics_dict["Deletions"] == self.total_words:
            return 0.0

        rnet_out = self.rnet(self.hs.view(1, -1), grad=False)

        score_tensor = torch.FloatTensor([self.data[2]]).to(DEVICE)
        loss = self.rnet.loss_fn(rnet_out.squeeze(-1), score_tensor)
        self.loss = loss.item()

        rnet_reward = np.log(1 - loss.item() + 1e-8)
        deletions_reward = (
            self.gamma * self.statistics_dict["Deletions"] / self.total_words
        )

        self.statistics_dict["RNet reward"] = rnet_reward
        self.statistics_dict["Deletions reward"] = deletions_reward

        return rnet_reward + deletions_reward

    def __init__(
        self,
        dataset: Dataset,
        srm: SRModel,
        rnet: RNet,
        gamma: float = ENV_GAMMA,
        random_sampling: bool = True,
    ) -> None:
        self.srm = srm
        self.rnet = rnet

        self.gamma = gamma

        self.dataset = dataset

        self.random_sampling = random_sampling
        self.idx = -1

        self.reset()

    def reset(self, idx: int = -1) -> torch.Tensor:
        self.loss = -1

        self.steps = 0

        self.sentence_idx = 0
        self.token_idx = 0

        if self.random_sampling:
            self.idx = np.random.randint(len(self.dataset))
        elif idx >= 0:
            self.idx = idx
        else:
            self.idx = (self.idx + 1) % len(self.dataset)

        self.data = self.dataset[self.idx]

        self.total_words = len(self.data[0]) + len(self.data[1])

        self.hs = torch.zeros((2, LSTM_LAYERS, LSTM_H_DIM)).to(DEVICE)
        self.cs = torch.zeros((2, LSTM_LAYERS, LSTM_H_DIM)).to(DEVICE)

        self.used_tokens: tuple[list[torch.Tensor], list[torch.Tensor], float] = (
            [],
            [],
            self.data[2].item(),
        )

        self.used_pure_tokens: tuple[list[str], list[str]] = ([], [])
        self.deleted_tokens_dict = {}

        self.statistics_dict = {
            "Initial Target length": len(self.data[0]),
            "Initial Candidate length": len(self.data[1]),
            "Processed Target length": 0,
            "Processed Candidate length": 0,
            "Deletions": 0,
            "RNet reward": 0.0,
            "Deletions reward": 0.0,
            "Deletions ratio": 0.0,
        }

        return self.get_state()

    def get_state(self) -> torch.Tensor:
        return self._get_state()

    def is_terminal(self) -> bool:
        return self.sentence_idx == 1 and self.token_idx == (len(self.data[1]) - 1)

    def interact(self, action: int) -> tuple[torch.Tensor, float, bool]:
        # 0 - retain
        # 1 - delete

        if self.is_terminal():
            return self._get_state(), 0, self.is_terminal()

        if action == 1:
            self.statistics_dict["Deletions"] += 1
            pure_token = self.data[self.sentence_idx + 3][self.token_idx].lower()
            self.deleted_tokens_dict[pure_token] = (
                self.deleted_tokens_dict.get(pure_token, 0) + 1
            )
            self.used_pure_tokens[self.sentence_idx].append("DELETED")

        elif action == 0:
            pure_token = self.data[self.sentence_idx + 3][self.token_idx].lower()
            self.used_pure_tokens[self.sentence_idx].append(pure_token)

            if self.sentence_idx == 0:
                self.statistics_dict["Processed Target length"] += 1
            else:
                self.statistics_dict["Processed Candidate length"] += 1

            token = self.data[self.sentence_idx][self.token_idx]
            self.used_tokens[self.sentence_idx].append(  # type: ignore
                token.clone().detach()
            )

            h_c = (
                self.hs[self.sentence_idx].clone().detach(),
                self.cs[self.sentence_idx].clone().detach(),
            )

            _, (h, c) = self.srm(token.view(1, -1), h_c, grad=False)
            self.hs[self.sentence_idx] = h.clone().detach()
            self.cs[self.sentence_idx] = c.clone().detach()

        self.steps += 1
        self.token_idx += 1
        if self.sentence_idx == 0 and self.token_idx >= len(self.data[0]):
            self.sentence_idx = 1
            self.token_idx = 0

        return self._get_state(), self._get_reward(), self.is_terminal()

    def get_used_tokens(self) -> tuple[list[torch.Tensor], list[torch.Tensor], float]:
        if len(self.used_tokens[0]) == 0:
            self.used_tokens[0].append(torch.zeros(LSTM_LAYERS, LSTM_H_DIM).to(DEVICE))
        if len(self.used_tokens[1]) == 0:
            self.used_tokens[1].append(torch.zeros(LSTM_LAYERS, LSTM_H_DIM).to(DEVICE))
        return self.used_tokens

    def get_observation_shape(self) -> int:
        return 2 * LSTM_LAYERS * LSTM_H_DIM + EMBED_DIM

    def get_actions_shape(self) -> int:
        return 2

    @staticmethod
    def sample_action() -> int:
        return np.random.choice([0, 1])

## DQN


In [61]:
class DQN(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int = 16) -> None:
        super(DQN, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.net(x)

## Agent

In [62]:
class Agent:
    def __init__(self) -> None:
        epsilon_fc = EPSILON_STRATEGY_DICT[EPSILON_STRATEGY]
        self.get_epsilon = partial(
            epsilon_fc,
            epsilon_start=EPSILON_START,
            epsilon_end=EPSILON_END,
            epsilon_const=EPSILON_CONSTANT,
            last_epoch=EPOCHS,
        )

    @torch.no_grad()
    def choose_optimal_action(self, state: torch.Tensor, dqn: nn.Module) -> int:
        q_vals_v = dqn(state)
        act_v = torch.argmax(q_vals_v)
        return int(act_v.item())

    def choose_action(self, state: torch.Tensor, dqn: nn.Module, epoch: int) -> int:
        if np.random.random() < self.get_epsilon(epoch):
            return Env.sample_action()
        return self.choose_optimal_action(state, dqn)

## Evaluation

In [63]:
def evaluate(
    srm: SRModel,
    rnet: RNet,
    dqn: nn.Module,
    agent: Agent,
    eval_env: Env,
) -> tuple[list[str], list[str]]:
    srm.net.eval()
    rnet.net.eval()
    dqn.eval()

    total_len = len(eval_env.dataset)

    losses = []

    iteration = 0
    state = eval_env.reset(0)

    targets = []
    candidates = []

    with tqdm(total=total_len, desc="Evaluation") as loop:
        while iteration < total_len:
            action = agent.choose_optimal_action(state, dqn)
            state2, _, done = eval_env.interact(action)
            state = state2.clone().detach()

            if done:
                iteration += 1
                losses.append(eval_env.loss)

                t, c = eval_env.used_pure_tokens
                targets.append(t)
                candidates.append(c)

                state = eval_env.reset()

                loop.update(1)

    return targets, candidates

## Analysis

In [75]:
initial_targets, initial_candidates = zip(*[(x[3], x[4]) for x in test_data])

itl = np.mean([len(x) for x in initial_targets])
icl = np.mean([len(x) for x in initial_candidates])

print(f"Mean init target len = {itl:.2f}")
print(f"Mean init candidate len = {icl:.2f}")

Mean init target len = 38.65
Mean init candidate len = 40.93


In [65]:
load_from_experiments(EXPERIMENT_NAME)

rnet = RNet(lr=RNET_LR)
srm = SRModel(lr=SRM_LR)

env = Env(test_data, srm, rnet, random_sampling=False)
agent = Agent()

dqn_net = DQN(
    input_dim=env.get_observation_shape(), output_dim=env.get_actions_shape()
).to(DEVICE)
dqn_target_net = DQN(
    input_dim=env.get_observation_shape(), output_dim=env.get_actions_shape()
).to(DEVICE)


load_model("dqn", dqn_net)
load_model("rnet", rnet.net)
load_model("srm", srm.net)

dqn_target_net.load_state_dict(dqn_net.state_dict())

{'RL_GAMMA': 1.0, 'GLOVE_DIM': 300, 'TRAIN_SIZE': 'md', 'TEST_SIZE': 'sm', 'EMBED_DIM': 300, 'LSTM_LAYERS': 1, 'LSTM_H_DIM': 300, 'RNET_DROPOUT': 0.5, 'ENV_GAMMA': 0.1, 'DQN_LR': 0.005, 'RNET_LR': 0.005, 'SRM_LR': 0.005, 'DQN_SYNC_PERIOD': 2, 'DQN_CLIP_GRAD': 0.0, 'PRETRAIN_SRM_RNET_EPOCHS': 200, 'PRETRAIN_DQN_EPOCHS': 100, 'EPISODES_BATCH': 10, 'PRETRAIN_SRM_RNET_BATCH': 10, 'EPSILON_START': 1.0, 'EPSILON_END': 0.01, 'EPSILON_STRATEGY': 'cos', 'EPSILON_CONSTANT': 50, 'FEATURES': '', 'COMMENTS': ''}


<All keys matched successfully>

In [95]:
targets, candidates = evaluate(srm, rnet, dqn_net, agent, env)

Evaluation: 100%|██████████| 147/147 [00:10<00:00, 13.72it/s]


In [76]:
tl = np.mean([len([w for w in x if w != "DELETED"]) for x in targets])
cl = np.mean([len([w for w in x if w != "DELETED"]) for x in candidates])

print(f"Mean target len = {tl:.2f} | {itl-tl:.2f} ~ {100-tl/itl*100:.2f}% deleted")
print(f"Mean candidate len = {cl:.2f} | {icl-cl:.2f} ~ {100-cl/icl*100:.2f}% deleted")

Mean target len = 27.56 | 11.09 ~ 28.69% deleted
Mean candidate len = 27.78 | 13.16 ~ 32.14% deleted


In [ ]:
for idx, (it, t) in enumerate(zip(initial_targets, targets)):
    print(idx, " ".join(it))
    print(idx, " ".join(t))
    print()

In [ ]:
for idx, (it, t) in enumerate(zip(initial_candidates, candidates)):
    print(idx, " ".join(it))
    print(idx, " ".join(t))
    print()

In [ ]:
# targets: 37,
# candidates: 51,

## Custom data

In [102]:
custom_data = PlagiarismDataset(
    [(" ".join(initial_targets[10]), "the", 0)],
)

custom_env = Env(custom_data, srm, rnet, random_sampling=False)

c_targets, _ = evaluate(srm, rnet, dqn_net, agent, custom_env)
c_targets